<a href="https://www.kaggle.com/code/sahidvelji/your-favourite-notebooks-sorted-by-date-upvoted?scriptVersionId=98954341" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Your favourite notebooks sorted by date upvoted

Since there is currently no option to sort notebooks by date upvoted in the "Favorites" tab, I decided that it wouldn't take long for me to implement this using [Meta Kaggle](https://www.kaggle.com/kaggle/meta-kaggle).

In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [2]:
import numpy as np
import pandas as pd
from IPython.display import display, HTML, Markdown
from pathlib import Path
        
KAGGLE_BASE_URL = "https://www.kaggle.com/"
PERFORMANCE_TIERS = {0: 'Novice',
                     1: 'Contributor',
                     2: 'Expert',
                     3: 'Master',
                     4: 'Grandmaster',
                     5: 'Staff'
                    }
MEDAL_TIERS = {1: 'golds',
               2: 'silvers',
               3: 'bronzes'
              }
medals = {i: f'<img style="float:left;padding-right:2px" src="https://kaggle.com/static/images/medals/notebooks/{MEDAL_TIERS[i]}@1x.png">' for i in range(1, 4)}
avatiers = {i: f'<img src="https://kaggle.com/static/images/avatier/avatier-{PERFORMANCE_TIERS[i].lower()}@2x.png">' for i in range(6)}

In [3]:
path = Path('/kaggle/input/meta-kaggle')

kernels = pd.read_csv(path/'Kernels.csv', 
                      parse_dates=['CreationDate', 'MadePublicDate', 'MedalAwardDate'],
                      infer_datetime_format=True
                     )
kernel_versions = pd.read_csv(path/'KernelVersions.csv', 
                              parse_dates=['CreationDate'],
                              infer_datetime_format=True
                             )
kernel_votes = pd.read_csv(path/'KernelVotes.csv', 
                           parse_dates=['VoteDate'], infer_datetime_format=True
                          )

users = pd.read_csv(path/'Users.csv',
                    parse_dates=['RegisterDate'], 
                    infer_datetime_format=True
                   )

To use this notebook:
1. Fork this notebook by clicking on `Copy and Edit`.
1. Set the username variable to your own username.
1. Select sorting and filtering options below and run the notebook.

In [4]:
username = 'sahidvelji'

MOST_RECENT_FIRST = True
INCLUDE_SELF_VOTES = True
NUM_RESULTS = 20

In [5]:
userid, displayname = users.loc[users['UserName'].eq(username), ['Id', 'DisplayName']].iloc[0]

df = (kernel_votes
      .loc[kernel_votes['UserId'].eq(userid), ['KernelVersionId', 'VoteDate']]
      .merge(kernel_versions[['Id', 'ScriptId', 'Title']], left_on='KernelVersionId', right_on='Id')
      .merge(kernels, left_on='ScriptId', right_on='Id')
      .merge(users, left_on='AuthorUserId', right_on='Id')
      .sort_values('VoteDate', ascending=not MOST_RECENT_FIRST)
      .pipe(lambda x: x.loc[x['UserName'].ne(username)] if not INCLUDE_SELF_VOTES else x)
      .head(NUM_RESULTS)
      .replace({'PerformanceTier': avatiers, 'Medal': medals})
      .assign(URL=lambda x: x['Medal'].fillna('') + f'<a href="{KAGGLE_BASE_URL}' + x['UserName'] + '/' + x['CurrentUrlSlug'] + '">' + x['Title'] + '</a>',
              User=lambda x: f'<a href="{KAGGLE_BASE_URL}' + x['UserName'] + '">' + x['DisplayName'] + '</a>' + x['PerformanceTier'])
      .reindex(columns=['VoteDate', 'TotalVotes', 'User', 'URL', 'TotalComments'])
     )

display(Markdown(f'# Notebooks upvoted by {displayname}'))
HTML(df.to_html(escape=False, index=False).replace('<td>', '<td style="text-align:left">'))

# Notebooks upvoted by Sahid Velji

VoteDate,TotalVotes,User,URL,TotalComments
2022-06-20,450,Jeremy Howard,Iterate like a grandmaster!,30
2022-06-18,19,Sahid Velji,Self-follows on Kaggle,2
2020-10-13,1293,Vopani,Tutorial on reading large datasets,106
2020-10-06,31,Raghawendra Singh,PANDA_medium_resolution_dataset_25x256x256,9
2020-09-06,362,Y.Nakama,MoA / PyTorch NN starter,26
2020-09-04,227,DanB,What is Log Loss?,25
2020-08-31,14,Sahid Velji,Your favourite notebooks sorted by date upvoted,5
2020-08-31,12,James Trotman,Discussion Hotness Plots,2
2020-08-28,83,Ruchi Bhatia,Movies on streaming platforms-EDA using Tableau,4
2020-08-25,10,Sahid Velji,MNIST digits with PyTorch,1
